## Monodelphis Domestica Single Cell RNA Seq
This notebook contains code to process the raw cellranger output from 10X Chromium single-cell RNA sequencing of adult opossum whole testis.

**by Daniel Stadtmauer and Kira Marshall**

Biological samples:

- Three adult replicates

## Software Import

These commands load the required packages into Python.

In [ ]:
import scanpy as sc

In [ ]:
import scprep
#import velocyto
import sccoda
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import phate
import magic
import os, tarfile
import anndata
#import scanpy as sc
import umap
import gseapy
import seaborn as sns
# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import scvelo as scv
import anndata2ri
import graphtools as gt
import gc

import harmonypy

The custom functions for this project, mainly cell-cell interaction modeling but also gene lookup, are for the time being stored in an experimental package [`chinpy`](https://gitlab.com/dnjst/chinpy).

In [ ]:
import chinpy

In [ ]:
%matplotlib inline

Next we set the scanpy figure parameters for how the plotting output will appear.
two options below

In [ ]:
sc.set_figure_params(scanpy=True, dpi=300, dpi_save=300, frameon=True, vector_friendly=True, fontsize=14, figsize=None, color_map=None, format='svg', facecolor=None, transparent=True, ipython_format='png2x')

In [ ]:
sc.settings.set_figure_params(dpi=80, dpi_save=2000, facecolor=None, transparent=True, format="png")

## Data Processing from 10X- AnnData
Read in the count matrix into an AnnData object, which holds many slots for annotations and different representations of the data. It also comes with its own HDF5-based file format: .h5ad.

this is from the scanpy tutorial page
https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html

In [ ]:
# load adata from files (three replicates post-SoupX)
data_batches_filepaths = ['rep1_outs_strainedCounts', 'rep2_outs_strainedCounts', 'rep3_outs_strainedCounts']

adata_batches = []

for i in data_batches_filepaths:
    adata_batches.append(sc.read_10x_mtx(i, var_names='gene_ids')) 
    
data_batches_labels = ['Rep1', 'Rep2', 'Rep3']

for i in range(len(adata_batches)):
    adata_batches[i].obs['sample_labels'] = data_batches_labels[i]
    

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
results_file = 'write/MdScRNA_3reps.h5ad'  # the file that will store the analysis results

## QC Calculations

Single-cell tutorials usually use the `MT-` prefix to detect mitochondrial genes, but this doesn't work for opossum because it doesn't have the MT nomenclature.



In [ ]:
newnames_dict = chinpy.pp.remap(dataset='mdomestica_gene_ensembl', longnames=True, only11=False, onlyhs=False, host='http://may2021.archive.ensembl.org', return_dict=True)

In [ ]:
adata_batches[0].var['Gene_ID']=adata_batches[0].var_names
adata_batches[1].var['Gene_ID']=adata_batches[1].var_names
adata_batches[2].var['Gene_ID']=adata_batches[2].var_names

# rename adata vars to scprep style
for itemj in range(len(adata_batches)):
    newnames, newnicks = [], []
    for itemi in adata_batches[itemj].var_names:
        newnames.append(newnames_dict[itemi])
        newnicks.append(newnames_dict[itemi].split(" (")[0])
    adata_batches[itemj].var_names = newnames
    adata_batches[itemj].var['gene_names'] = newnicks
    adata_batches[itemj].var['gene_names_full'] = newnames

In [ ]:
# grab mitochdonrial (MT) gene list from biomart based upon being on the opossum "MT" chromosome 
from pybiomart import Server
server = Server(host='http://may2021.archive.ensembl.org')
dataset = (server.marts['ENSEMBL_MART_ENSEMBL'].datasets['mdomestica_gene_ensembl'])

mtgenelist = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name'], filters={'chromosome_name': ['MT']})
mtgenelist = mtgenelist['Gene stable ID'].to_list()

# more extensive list based upon homology to any gene on the human "MT" chromosome 

mtgenelist2 = server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl'].query(attributes=['ensembl_gene_id', 'mdomestica_homolog_ensembl_gene', 'external_gene_name', 'chromosome_name', 'mdomestica_homolog_orthology_type'])
mtgenelist2 = mtgenelist2[mtgenelist2['Chromosome/scaffold name'] == "MT"]['Opossum gene stable ID']
mtgenelist2 = mtgenelist2[~pd.isnull(mtgenelist2)]
mtgenelist = list(mtgenelist2)

# get full gene name
mtgenelist_named = [newnames_dict[itemi] for itemi in mtgenelist]

# subset to just the ones in this data set
mtgenelist_named_cut = [x for x in mtgenelist_named if x in adata_batches[0].var_names]

# calculate MT and Ribo-related QC

for i in range(len(adata_batches)):
    #sc.pl.highest_expr_genes(adata, n_top=20, ) # plot top genes
    adata_batches[i].var['mt'] = [itemi in mtgenelist_named for itemi in adata_batches[i].var_names] #adata_batches[i].var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    #adata_batches[i].var['mt'] = adata_batches[i].var_names.str.startswith('MT-')
    adata_batches[i].var['ribo'] = adata_batches[i].var_names.str.startswith('RPS' or 'RPL' or 'MRPS' or 'MRPL')  # annotate the group of mitochondrial genes as 'mt' 
    sc.pp.calculate_qc_metrics(adata_batches[i], qc_vars=('ribo','mt'), percent_top=None, log1p=False, inplace=True)
    #sc.pl.violin(adata_batches[i], ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True, stripplot=False, color="red")
    #sc.pl.scatter(adata_batches[i], x='total_counts', y='pct_counts_mt')
    #sc.pl.scatter(adata_batches[i], x='total_counts', y='pct_counts_ribo')
    #sc.pl.scatter(adata_batches[i], x='total_counts', y='n_genes_by_counts')

In [ ]:
len(mtgenelist)

In [ ]:
adata_batches[i].var
sum(adata_batches[i].var['ribo'])

In [ ]:
# plot joint diagrams

fig, axes = plt.subplots(1,3, figsize=(5*4,5*3))
#for ax, i in zip(axes.flatten(), range(len(adata_batches))):
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.scatter(adata_batches[i], x='total_counts', y='pct_counts_mt', show=False, ax=ax)
    ax.set_title(data_batches_labels[i] + ")")
    
fig.tight_layout()

## Visualizing QC Metrics

looking at the QC metrics as plots allows us to see where thresholds should be placed.

First, looking at the stats together

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'n_genes_by_counts', log=False, jitter=0.4, stripplot=False, show=False, ax=ax, color="teal")
    ax.set_title(data_batches_labels[i])
    
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'total_counts', log=True, jitter=0.4, stripplot=False, show=False, ax=ax)
    ax.set_title(data_batches_labels[i])
    #ax.set_ylim([0, 1e5]) # standardize all limits
    
fig.tight_layout()

In [ ]:
# plot joint diagrams

fig, axes = plt.subplots(1,3, figsize=(3*4,8))
#for ax, i in zip(axes.flatten(), range(len(adata_batches))):
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.scatter(adata_batches[i], x='total_counts', y='n_genes_by_counts', show=False, ax=ax)
    ax.set_title(data_batches_labels[i])
    
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'pct_counts_mt', log=False, jitter=0.4, stripplot=False, show=False, ax=ax, color="teal")
    ax.set_title(data_batches_labels[i])
    
fig.tight_layout()

fig, axes = plt.subplots(1,3, figsize=(3*4,8))
#for ax, i in zip(axes.flatten(), range(len(adata_batches))):
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.scatter(adata_batches[i], x='total_counts', y='pct_counts_mt', show=False, ax=ax)
    ax.set_title(data_batches_labels[i])
    
fig.tight_layout()

fig, axes = plt.subplots(1,3, figsize=(3*4,8))
#for ax, i in zip(axes.flatten(), range(len(adata_batches))):
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.scatter(adata_batches[i], x='total_counts', y='pct_counts_ribo', show=False, ax=ax)
    ax.set_title(data_batches_labels[i])
    
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'n_genes_by_counts', log=False, jitter=0.4, stripplot=True, show=False, ax=ax)
    ax.set_title(data_batches_labels[i])
    ax.axhline(y=np.percentile(adata_batches[i].obs['n_genes_by_counts'], 15), color="red")
    ax.axhline(y=np.percentile(adata_batches[i].obs['n_genes_by_counts'], 98), color="red")
    ax.set_ylim([0, 7000])
    
fig.tight_layout()

In [ ]:
np.percentile(adata_batches[1].obs['n_genes_by_counts'], 98)

In [ ]:
adata_batches[0].shape

In [ ]:
adata_batches[1].shape

In [ ]:
adata_batches[2].shape

In [ ]:
adata_batches

## Cutoff step

instead of removing -MT, remove >98th percentile and <15th percentile

In [ ]:
rep_tables = adata_batches

In [ ]:
adata_batches[0] = adata_batches[0][(adata_batches[0].obs['n_genes_by_counts'] < np.percentile(adata_batches[0].obs['n_genes_by_counts'], 98)) & (adata_batches[0].obs['n_genes_by_counts'] >= np.percentile(adata_batches[0].obs['n_genes_by_counts'], 15))]

In [ ]:
adata_batches[1] = adata_batches[1][(adata_batches[1].obs['n_genes_by_counts'] < np.percentile(adata_batches[1].obs['n_genes_by_counts'], 98)) & (adata_batches[1].obs['n_genes_by_counts'] >= np.percentile(adata_batches[1].obs['n_genes_by_counts'], 15))]

In [ ]:
adata_batches[2] = adata_batches[2][(adata_batches[2].obs['n_genes_by_counts'] < np.percentile(adata_batches[2].obs['n_genes_by_counts'], 98)) & (adata_batches[2].obs['n_genes_by_counts'] >= np.percentile(adata_batches[2].obs['n_genes_by_counts'], 15))]

In [ ]:
#check data shape after cutoff step
adata_batches

In [ ]:
adata_batches[0].shape

In [ ]:
adata_batches[1].shape

In [ ]:
adata_batches[2].shape

## Doublet Identification

### DoubletDetection

This software [`DoubletDetection`](https://pypi.org/project/doubletdetection/) predicts which cells are doublets. The advantage to this method is that it does not require an *a priori* prediction of how many doublets will be found, and instead runs iteratively until the model converges to an answer.

In [ ]:
os.mkdir(os.path.join(os.getcwd(), "outs"))
os.mkdir(os.path.join(os.getcwd(), "outs", "DoubletDetection"))

In [ ]:
os.path.exists(os.path.abspath(os.path.join(os.getcwd(), "outs", "DoubletDetection")))

In [ ]:
import doubletdetection

for i in range(len(adata_batches)):
    
    clf = doubletdetection.BoostClassifier(
    n_iters=25,  
    standard_scaling=True)
    doublets = clf.fit(adata_batches[i].X).predict(p_thresh=1e-16, voter_thresh=0.5)
    doublet_score = clf.doublet_score()
    
    # mkdir for output if not exist already
    outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "DoubletDetection"))
    if os.path.exists(outdir) == False:
        os.makedirs(outdir)
    # write plots
    f = doubletdetection.plot.convergence(clf, save=os.path.abspath(os.path.join(outdir, 'convergence_test_' + data_batches_labels[i] + '.pdf')), show=True, p_thresh=1e-16, voter_thresh=0.5)
    f3 = doubletdetection.plot.threshold(clf, save=os.path.abspath(os.path.join(outdir, 'threshold_test_' + data_batches_labels[i] + '.pdf')), show=True, p_step=6)
    
    # count doublets
    print("detected " + str(sum(doublets)) + " doublets in " + data_batches_labels[i])
    
    # apply
    adata_batches[i].obs['doublet_score_dd'] = doublet_score
    adata_batches[i].obs['doublet_dd'] = doublets

## Remove Doublets

In [ ]:
# filter cells thought to be doublets

for i in range(len(adata_batches)):
    adata_batches[i] = adata_batches[i][(adata_batches[i].obs['doublet_dd'] == 0.0)]

In [ ]:
adata_batches[0]

In [ ]:
adata_batches[1]

In [ ]:
adata_batches[2]

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'n_genes_by_counts', log=False, jitter=0.4, stripplot=False, show=False, ax=ax)

fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'total_counts', log=False, jitter=0.4, stripplot=False, show=False, ax=ax)
    ax.set_ylim([0, 20000])

fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'total_counts', log=False, jitter=0.4, stripplot=False, show=False, ax=ax)

fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'total_counts', log=True, jitter=0.4, stripplot=True, show=False, ax=ax)

fig, axes = plt.subplots(1,3, figsize=(3*4,8))
for i, ax in zip(range(len(adata_batches)), axes.flatten()):
    sc.pl.violin(adata_batches[i], 'total_counts', log=True, jitter=0.4, stripplot=True, show=False, ax=ax)
    ax.set_ylim([0, 1000])

fig.tight_layout()

## Merge

This uses [anndata concatenation](https://anndata.readthedocs.io/en/latest/concatenation.html) to concatenate anndata objects from independent samples.

In [ ]:
# concatenate all anndata NO VELOC
adata = anndata.concat(adata_batches, keys=data_batches_labels, index_unique="_", merge="same")

# Normalization

The approach to normalization will be to do the standard log(x+1), which will be used for DGE and statistical testing since many of the single-cell applications assume this method.

But, we'll also store [layers](https://anndata.readthedocs.io/en/latest/anndata.AnnData.layers.html) on the anndata object with square-root transcripts per million and MAGIC-denoised square-root transcripts per million for visualization purposes. Layers are additional matrices of identical size to the cell-gene matrix but different values, for instance "raw" and normalized counts.

De-noising of the data using MAGIC is best for analyses that examine single genes, such as visualizing a gene's expression across cells or clusters

In [ ]:
#re-transforming if needed
sc.pp.log1p(adata)

In [ ]:
# save raw counts to layer

adata.layers["counts"] = adata.X.copy()

# scprep normalization

# may want to use adata.to_df() to turn adata.X into a regular matrix?

data_norm, library_size = scprep.normalize.library_size_normalize(adata.to_df(), rescale=1e6, return_library_size=True)
data_sqrt = scprep.transform.sqrt(data_norm)
data_magic = magic.MAGIC(random_state = 0).fit_transform(data_sqrt)
adata.obs['scprep_library_size'] = library_size

# add scprep SQRT CPM and MAGIC layers

adata.layers['sqrtcpm'] = data_sqrt
adata.layers['sqrtcpm_magic'] = data_magic

# scanpy normalization

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
#sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

# save the lognorm data as .raw for statistical testing purposes
#adata.raw = adata

#sc.pl.highly_variable_genes(adata)

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

## Data Integration and Embedding


## Harmony

We use Haromny as implemented in scanpy using [`scanpy.external.pp.harmony_integrate`](https://scanpy.readthedocs.io/en/stable/generated/scanpy.external.pp.harmony_integrate.html).

In [ ]:
sc.tl.pca(adata, random_state=0)
sc.external.pp.harmony_integrate(adata, key = 'sample_labels', random_state=0)
sc.pl.pca_variance_ratio(adata, log=True)

We can assess what `harmony` did by comparing the first two PCA dimensions before and after running it.

In [ ]:
# plot PCA versus harmony-corrected PCA

fig, axes = plt.subplots(1,2, figsize=(5*2,4*1))

sc.pl.embedding(adata, "X_pca", title="Pre Harmony", legend_loc="right margin", legend_fontsize="xx-small", show=False, ax=axes[0])
sc.pl.embedding(adata, "X_pca_harmony", title="Post Harmony", legend_loc="right margin", legend_fontsize="xx-small", show=False, ax=axes[1])

fig.tight_layout()

# plot PCA versus harmony-corrected PCA

fig, axes = plt.subplots(1,2, figsize=(5*2,4*1))

sc.pl.embedding(adata, "X_pca", color="sample_labels", title="Pre Harmony", legend_loc="right margin", legend_fontsize="xx-small", show=False, ax=axes[0])
sc.pl.embedding(adata, "X_pca_harmony", color="sample_labels", title="Post Harmony", legend_loc="right margin", legend_fontsize="xx-small", show=False, ax=axes[1])
#plt.savefig("Md_Harmony.png")
fig.tight_layout()



## Embedding

The goal of embedding is to visualize the cells in 2 dimensions and identify clusters of cell types and states.

The most common method we will use first is `UMAP`.


In [ ]:
# calculate neighbors graph

#regular
sc.pp.neighbors(adata, use_rep = "X_pca", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors")
#harmony
sc.pp.neighbors(adata, use_rep = "X_pca_harmony", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors_harmony")

# use harmony's 'neighbors' key to embed UMAP
sc.tl.umap(adata, neighbors_key="neighbors_harmony", random_state=0)

# embed PHATE

sc.external.tl.phate(adata, k=5, a=20, t=150, random_state=0)

In [ ]:
#plot umap colored by replicate
sc.pl.umap(adata, color="sample_labels", palette=['lightgrey', 'mediumaquamarine', 'teal',])

In [ ]:
#Plot each replicate individually
sc.pl.umap(adata[adata.obs['sample_labels'] == 'Rep1'])
sc.pl.umap(adata[adata.obs['sample_labels'] == 'Rep2'])
sc.pl.umap(adata[adata.obs['sample_labels'] == 'Rep3'])

In [ ]:
sc.pl.scatter(adata, basis='phate')


In [ ]:
#Phate plot colored by replicate
fig, axes = plt.subplots(1,2, figsize=(5*2,4*1))
sc.pl.scatter(adata, basis="phate", color="sample_labels", title="PHATE", legend_loc="right margin", legend_fontsize="xx-small", show=False, ax=axes[1])


# Cluster Analysis

## Leiden

Leiden clustering is a good starting point for UMAP embeddings. It's a graph-based clustering and takes a resolution parameter, the defualt of which is 1.

In [ ]:
resolutions = (0.4, 0.6, 1.0, 1.2, 1.4, 1.8)
for resolution in resolutions:
    sc.tl.leiden(adata, resolution=resolution, key_added="leiden_" + str("{:.1f}".format(resolution)), neighbors_key="neighbors_harmony", random_state=0)

In [ ]:
# plot all resolutions
fig, axes = plt.subplots(2,3, figsize=(4*3,4*2))
for j, ax in zip(resolutions, axes.flatten()):
    myslice = "leiden_" + str("{:.1f}".format(j))
    sc.pl.umap(adata, color=myslice, title=myslice + ": " + str(len(set(adata.obs[myslice]))) + " clusters", legend_loc="on data", show=False, ax=ax)
    #plt.savefig("Md_AllData_LeidenClusters.png")
fig.tight_layout()

In [ ]:
#set slice to the one you will be using
myslice = "leiden_1.4"

# conduct the 3 main tests on cluster IDs
sc.tl.rank_genes_groups(adata, "leiden_1.4", method='logreg', key_added="rank_genes_groups_logreg")
sc.tl.rank_genes_groups(adata, "leiden_1.4", method='t-test', key_added="rank_genes_groups_t-test")
sc.tl.rank_genes_groups(adata, "leiden_1.4", method='wilcoxon', key_added="rank_genes_groups_wilcoxon")

In [ ]:
#Write Cluster Markers

myslice = "leiden_1.4"
bonus = "Opossum_Original_Comb_"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:
    # Logistic regression marker identification
    result = adata.uns['rank_genes_groups_logreg']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'scores']}).head(500)
    markers.to_excel(writer, sheet_name = "markers_all_logreg")
    
    # t-test marker identification
    result = adata.uns['rank_genes_groups_t-test']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(500)
    markers.to_excel(writer, sheet_name = "markers_all_ttest")
    
    # Wilcoxon marker identification
    result = adata.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(500)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

# Marker Gene Lists

In [ ]:
marker_genes_comb_germ = ['CRABP1 (ENSMODG00000011710)', 'SOHLH1 (ENSMODG00000029683)', 
                'KIT (ENSMODG00000020671)', 'DMRT1 (ENSMODG00000003401)',  'UCHL1 (ENSMODG00000020568)',
                'STRA8 (ENSMODG00000014081)', 'SYCP1 (ENSMODG00000004800)', 'SYCP2 (ENSMODG00000016617)', 
                'CCNA1 (ENSMODG00000009993)', 'AURKA (ENSMODG00000016435)', 
                'ZPBP (ENSMODG00000009302)', 'TMEM144 (ENSMODG00000001956)', 
                'SPACA1 (ENSMODG00000018292)', 'TNP1 (ENSMODG00000024866)', 'TSSK6 (ENSMODG00000023787)', 
                'PRM1 (ENSMODG00000005135)']

marker_genes_comb_som = ['DCN (ENSMODG00000009654)', 'CD74 (ENSMODG00000028756)',  
                'TYROBP (ENSMODG00000014057)', 'SOX9 (ENSMODG00000006526)', 'FSHR (ENSMODG00000001267)',
                'CLU (ENSMODG00000015960)', 'STAR (ENSMODG00000010748)', 'CYP17A1 (ENSMODG00000011205)', 
                'CYP11A1 (ENSMODG00000009689)', 'ACTA2 (ENSMODG00000007394)', 'TAGLN (ENSMODG00000013744)', 
                'PECAM1 (ENSMODG00000025673)',
                'RAMP2 (ENSMODG00000015038)']


In [ ]:
# plot marker genes-germ
fig, axes = plt.subplots(13,3, figsize=(4*3,4*13))
for j, ax in zip(marker_genes_comb_germ, axes.flatten()):
    sc.pl.umap(adata, color=chinpy.pp.search(adata, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
    
fig.tight_layout()

In [ ]:
# plot marker genes-somatic
fig, axes = plt.subplots(7,3, figsize=(4*3,4*7))
for j, ax in zip(marker_genes_comb_som, axes.flatten()):
    sc.pl.umap(adata, color=chinpy.pp.search(adata, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
    
fig.tight_layout()

## Seeing if mitochondrial genes cluster somewhere

In [ ]:
# plot marker genes
fig, axes = plt.subplots(10,3, figsize=(4*3,4*10))
for j, ax in zip(mtgenelist, axes.flatten()):
    sc.pl.umap(adata, color=chinpy.pp.search(adata, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
    
fig.tight_layout()

# Cluster Labels

In [ ]:
#label clusters as germ or somatic
coarse_labels = {"0": "somatic_cells",
                "1": "somatic_cells",
                "2": "somatic_cells",
                "3": "somatic_cells",
                "4": "germ_cells",
                "5": "germ_cells",
                "6": "germ_cells",
                ...
                "21": "somatic_cells"}


#germ_cells
#somatic_cells


In [ ]:
#add cluster labels
adata.obs['leiden_coarse'] = [coarse_labels[i] for i in adata.obs['leiden_0.4']]



In [ ]:
sc.pl.umap(adata, color=["leiden_coarse"])
#, save = "Md_AllData_UMAP_CellCategory.png"

Saved adata as an h5ad so next time I can load this in as an object instead of repeating all steps and getting a different umap

In [ ]:
adata.write_h5ad('Md_SingleCell_3Rep.h5ad')

In [ ]:
# read h5ad back if needed
adata = anndata.read_h5ad("path/to/Md_SingleCell_3Rep.h5ad")


In [ ]:
adata_germ = adata[adata.obs["leiden_coarse"] == "germ_cells"]
adata_som = adata[adata.obs["leiden_coarse"] == "somatic_cells"]
adata_sub = adata[adata.obs["leiden_coarse"] != "remove"]

In [ ]:
adata

In [ ]:
adata_sub

In [ ]:
adata_som

In [ ]:
adata_germ

## Re-embedding

In [ ]:
sc.tl.pca(adata_sub, random_state=0, use_highly_variable=True)
# calculate neighbors graph

#regular
sc.pp.neighbors(adata_sub, use_rep = "X_pca", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors")
#harmony
sc.pp.neighbors(adata_sub, use_rep = "X_pca_harmony", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors_harmony")

# use harmony's 'neighbors' key to embed UMAP
sc.tl.umap(adata_sub, neighbors_key="neighbors_harmony", random_state=0)

# embed PHATE

sc.external.tl.phate(adata_sub, k=5, a=20, t=150, random_state=0)

In [ ]:
# plot UMAP 
sc.pl.umap(adata_sub)
sc.pl.scatter(adata_sub, basis='phate')

In [ ]:
#sc.pl.umap(adata_sub, color=["leiden_named"])
#sc.pl.umap(adata_sub, color=["leiden_named"]) #, save = "Md_Sub_UMAP_CellCategory.png")
sc.pl.umap(adata_sub, color=["leiden_coarse"] , palette = {
    "germ_cells": "black",
    "somatic_cells": "silver"
})
#, save = "Md_Sub_UMAP_CellCategory.png")
sc.pl.umap(adata_sub, color=["leiden_coarse"])
#sc.pl.umap(adata_sub, color=["leiden_coarse"], save = "Md_Sub_UMAP_CellCategory.png")

In [ ]:
sc.tl.pca(adata_germ, random_state=0, use_highly_variable=True)
# calculate neighbors graph

#regular
sc.pp.neighbors(adata_germ, use_rep = "X_pca", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors")
#harmony
sc.pp.neighbors(adata_germ, use_rep = "X_pca_harmony", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors_harmony")

# use harmony's 'neighbors' key to embed UMAP
sc.tl.umap(adata_germ, neighbors_key="neighbors_harmony", random_state=0)

# embed PHATE

sc.external.tl.phate(adata_germ, k=5, a=20, t=150, random_state=0)

In [ ]:
# plot UMAP 
sc.pl.umap(adata_germ)
sc.pl.scatter(adata_germ, basis='phate')

In [ ]:
sc.pl.umap(adata_germ, color="sample_labels", palette=['lightgrey', 'mediumaquamarine', 'teal'])

In [ ]:
sc.pl.scatter(adata_germ, basis='phate', color="sample_labels", palette=['lightgrey', 'mediumaquamarine', 'teal'])

In [ ]:
sc.pl.umap(adata_germ, color=["leiden_named"])

In [ ]:
sc.tl.pca(adata_som, random_state=0, use_highly_variable=True)
# calculate neighbors graph

#regular
sc.pp.neighbors(adata_som, use_rep = "X_pca", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors")
#harmony
sc.pp.neighbors(adata_som, use_rep = "X_pca_harmony", n_neighbors=10, n_pcs=40, random_state=0, key_added="neighbors_harmony")

# use harmony's 'neighbors' key to embed UMAP
sc.tl.umap(adata_som, neighbors_key="neighbors_harmony", random_state=0)

# embed PHATE

#sc.external.tl.phate(adata_som, k=5, a=20, t=150, random_state=0)

In [ ]:
# plot UMAP 
sc.pl.umap(adata_som, color="sample_labels", palette=['lightgrey', 'mediumaquamarine', 'teal'])
#sc.pl.scatter(adata_som, basis='phate')

## Cluster Markers in new subset clusters

In [ ]:
# plot germ marker genes in germ cells
fig, axes = plt.subplots(13,3, figsize=(4*3,4*13))
for j, ax in zip(marker_genes_comb_germ, axes.flatten()):
    sc.pl.umap(adata_germ, color=chinpy.pp.search(adata_germ, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
    
fig.tight_layout()

In [ ]:
# plot somatic marker genes in germ cells
fig, axes = plt.subplots(8,3, figsize=(4*3,4*8))
for j, ax in zip(marker_genes_comb_som, axes.flatten()):
    sc.pl.umap(adata_germ, color=chinpy.pp.search(adata_germ, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
#plt.savefig("Md_GermCell_MarkerGenesA.png")       
fig.tight_layout()

In [ ]:
# plot marker genes in somatic cells
fig, axes = plt.subplots(8,3, figsize=(4*3,4*8))
for j, ax in zip(marker_genes_comb_som, axes.flatten()):
    sc.pl.umap(adata_som, color=chinpy.pp.search(adata_som, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
    
fig.tight_layout()

In [ ]:
# plot marker genes in somatic cells
fig, axes = plt.subplots(13,3, figsize=(4*3,4*13))
for j, ax in zip(marker_genes_comb_germ, axes.flatten()):
    sc.pl.umap(adata_som, color=chinpy.pp.search(adata_som, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
    
fig.tight_layout()

## Re-Clustering with new subset data

In [ ]:
resolutions = (0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.8)
for resolution in resolutions:
    sc.tl.leiden(adata_sub, resolution=resolution, key_added="leiden_" + str("{:.1f}".format(resolution)), neighbors_key="neighbors_harmony", random_state=0)

In [ ]:
# plot all resolutions
fig, axes = plt.subplots(2,3, figsize=(4*3,4*2))
for j, ax in zip(resolutions, axes.flatten()):
    myslice = "leiden_" + str("{:.1f}".format(j))
    sc.pl.umap(adata_sub, color=myslice, title=myslice + ": " + str(len(set(adata_sub.obs[myslice]))) + " clusters", legend_loc="on data", show=False, ax=ax)
    #plt.savefig("Md_Sub_LeidenClusters.png")  
fig.tight_layout()

In [ ]:
resolutions = (0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.8)
for resolution in resolutions:
    sc.tl.leiden(adata_germ, resolution=resolution, key_added="leiden_" + str("{:.1f}".format(resolution)), neighbors_key="neighbors_harmony", random_state=0)

In [ ]:
# plot all resolutions
fig, axes = plt.subplots(2,3, figsize=(4*3,4*2))
for j, ax in zip(resolutions, axes.flatten()):
    myslice = "leiden_" + str("{:.1f}".format(j))
    sc.pl.umap(adata_germ, color=myslice, title=myslice + ": " + str(len(set(adata_germ.obs[myslice]))) + " clusters", legend_loc="on data", show=False, ax=ax)
#plt.savefig("Md_GermCell_LeidenClusters.png")   
fig.tight_layout()

In [ ]:
#This is removing the transition group so may need to address that early!

# plot all resolutions
fig, axes = plt.subplots(2,3, figsize=(4*3,4*2))
for j, ax in zip(resolutions, axes.flatten()):
    myslice = "leiden_" + str("{:.1f}".format(j))
    sc.pl.umap(adata_germ_notrans, color=myslice, title=myslice + ": " + str(len(set(adata_germ_notrans.obs[myslice]))) + " clusters", legend_loc="on data", show=False, ax=ax)
#plt.savefig("Md_GermCell_LeidenClusters.png")   
fig.tight_layout()

In [ ]:
resolutions = (0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.8)
for resolution in resolutions:
    sc.tl.leiden(adata_som, resolution=resolution, key_added="leiden_" + str("{:.1f}".format(resolution)), neighbors_key="neighbors_harmony", random_state=0)

In [ ]:
# plot all resolutions
fig, axes = plt.subplots(2,3, figsize=(4*3,4*2))
for j, ax in zip(resolutions, axes.flatten()):
    myslice = "leiden_" + str("{:.1f}".format(j))
    sc.pl.umap(adata_som, color=myslice, title=myslice + ": " + str(len(set(adata_som.obs[myslice]))) + " clusters", legend_loc="on data", show=False, ax=ax)
    plt.savefig("Md_SomCell_LeidenClusters_DevCell.png")
fig.tight_layout()

# Making UMAPs by replicate

In [ ]:
# plot clusters on UMAP -sub
sc.pl.scatter(adata_sub, basis="umap", color="sample_labels", title="", palette=sc.pl.palettes.zeileis_28, legend_loc="right margin", legend_fontsize=4) 
#, save = "Md_sub_reps.png")

In [ ]:
# plot clusters on UMAP -germ cells
sc.pl.scatter(adata_germ, basis="umap", color="sample_labels", title="", palette=sc.pl.palettes.zeileis_28, legend_loc="right margin", legend_fontsize=4) 
#, save = "Md_germ_reps.png")


In [ ]:
# plot clusters on UMAP
sc.pl.scatter(adata_som, basis="umap", color="sample_labels", title="", palette=sc.pl.palettes.zeileis_28, legend_loc="right margin", legend_fontsize=4) 
#, save = "Md_Som_reps_DevCell.png")


## Getting New Top Genes

In [ ]:
myslice = "leiden_0.6"

# conduct the 3 main tests on cluster IDs
sc.tl.rank_genes_groups(adata_som, "leiden_0.6", method='logreg', key_added="rank_genes_groups_logreg")
sc.tl.rank_genes_groups(adata_som, "leiden_0.6", method='t-test', key_added="rank_genes_groups_t-test")
sc.tl.rank_genes_groups(adata_som, "leiden_0.6", method='wilcoxon', key_added="rank_genes_groups_wilcoxon")

In [ ]:
# Write Cluster Markers

myslice = "leiden_0.6"
bonus = "Opossum_SomCells_Comb_"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:
    # Logistic regression marker identification
    result = adata_som.uns['rank_genes_groups_logreg']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'scores']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_logreg")
    
    # t-test marker identification
    result = adata_som.uns['rank_genes_groups_t-test']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_ttest")
    
    # Wilcoxon marker identification
    result = adata_som.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

In [ ]:
myslice = "leiden_1.4"

# conduct the 3 main tests on cluster IDs
sc.tl.rank_genes_groups(adata_germ, "leiden_1.4", method='logreg', key_added="rank_genes_groups_logreg")
sc.tl.rank_genes_groups(adata_germ, "leiden_1.4", method='t-test', key_added="rank_genes_groups_t-test")
sc.tl.rank_genes_groups(adata_germ, "leiden_1.4", method='wilcoxon', key_added="rank_genes_groups_wilcoxon")

In [ ]:
# Write Cluster Markers

myslice = "leiden_1.4"
bonus = "Opossum_GermCells_ThreeRep_"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:
    # Logistic regression marker identification
    result = adata_germ.uns['rank_genes_groups_logreg']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'scores']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_logreg")
    
    # t-test marker identification
    result = adata_germ.uns['rank_genes_groups_t-test']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_ttest")
    
    # Wilcoxon marker identification
    result = adata_germ.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

# Making PHATE plots with marker genes

In [ ]:
marker_genes_comb_germA_phate = ['DDX4 (ENSMODG00000019511)', 'LIN28A (ENSMODG00000014236)', 'ID4 (ENSMODG00000011307)', 'ID4 (ENSMODG00000011307)',
                'SALL1 (ENSMODG00000013114)', 'SALL4 (ENSMODG00000016387)', 'ZRSR2 (ENSMODG00000017202)', 'ZRSR2 (ENSMODG00000017202)',
                'GFRA1 (ENSMODG00000009606)', 'PIWIL4 (ENSMODG00000000217)', 'SOHLH1 (ENSMODG00000029683)', 'SOHLH1 (ENSMODG00000029683)', 
                'KIT (ENSMODG00000020671)', 'DMRT1 (ENSMODG00000003401)',  'UCHL1 (ENSMODG00000020568)', 'UCHL1 (ENSMODG00000020568)',
                'STRA8 (ENSMODG00000014081)', 'PBX3 (ENSMODG00000019785)', 'ENSMODG00000037413 (ENSMODG00000037413)', 'ENSMODG00000037413 (ENSMODG00000037413)'] 
                           
                           
marker_genes_comb_germB_phate = ['TOP2B (ENSMODG00000014954)', 'RASSF1 (ENSMODG00000050482)', 'PIWIL1 (ENSMODG00000015619)', 'PIWIL1 (ENSMODG00000015619)',
                'SYCP1 (ENSMODG00000004800)', 'SYCP2 (ENSMODG00000016617)', 'HORMAD1 (ENSMODG00000018807)', 'HORMAD1 (ENSMODG00000018807)',
                'HORMAD2 (ENSMODG00000009221)', 'TANK (ENSMODG00000005310)', 'ADAD2 (ENSMODG00000004888)', 'ADAD2 (ENSMODG00000004888)', 
                'MEIOB (ENSMODG00000016106)', 'DMC1 (ENSMODG00000009655)', 'CCNB1 (ENSMODG00000019699)', 'CCNB1 (ENSMODG00000019699)'] 
                           
marker_genes_comb_germC_phate = ['AURKA (ENSMODG00000016435)', 'ZPBP (ENSMODG00000009302)', 'ACOT9 (ENSMODG00000007768)', 'ACOT9 (ENSMODG00000007768)', 
                'ENSMODG00000001263 (ENSMODG00000001263)', 'TMEM144 (ENSMODG00000001956)', 'SPACA1 (ENSMODG00000018292)', 'SPACA1 (ENSMODG00000018292)', 
                'ENSMODG00000008215 (ENSMODG00000008215)', 'TNP1 (ENSMODG00000024866)', 'TSSK6 (ENSMODG00000023787)', 'TSSK6 (ENSMODG00000023787)',
                'PRM1 (ENSMODG00000005135)', 'NRBP1 (ENSMODG00000015637)', 'GABBR2 (ENSMODG00000001896)']



In [ ]:
# plot marker genes
fig, axes = plt.subplots(4,4, figsize=(4*4,4*4))
for j, ax in zip(marker_genes_comb_germC_phate, axes.flatten()):
    sc.pl.scatter(adata_germ, color=chinpy.pp.search(adata_germ, j, startswith=True), layers="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax, basis="phate")
    
fig.tight_layout()



# Assigning Cluster Labels

In [ ]:
#coarse labels
new_labels_germ = {"0": "Bpermatocyte",
                "1": "Spermatocyte",
                  ...
                "26": "ElongatingSpermatid"}
#'Spermatogonia', 'Spermatocyte', 'RoundSpermatid', 'ElongatingSpermatid',

In [ ]:
#fine labels
new_labels_germ_2 = {"0": "Spermatocyte",
                "1": "LateSpermatocyte",
                  ...
                "25": "LateSpermatid",
                "26": "ElongatingSpermatid"}

#'Spermatogonia', 'Spermatocyte', 'LateSpermatocyte', 'RoundSpermatid', 'ElongatingSpermatid', 'ElongatedSpermatid'

In [ ]:
adata_germ.obs['new_labels_germ'] = [new_labels_germ[i] for i in adata_germ.obs['leiden_1.4']]
adata_germ.obs['new_labels_germ_2'] = [new_labels_germ_2[i] for i in adata_germ.obs['leiden_1.4']]

In [ ]:
sc.pl.umap(adata_germ, color="new_labels_germ", palette = {
         "Spermatogonia": "palegreen",
        "Spermatocyte": "darkgreen",
        "RoundSpermatid": "steelblue",
        "ElongatingSpermatid": "indigo"
    }) #, save = "MdGermClusters.png")

In [ ]:
sc.pl.umap(adata_germ, color="new_labels_germ_2", palette = {
        "Spermatogonia": "palegreen",
        "Spermatocyte": "mediumseagreen",
        "LateSpermatocyte": "darkgreen",   
        "SpermatocyteToSpermatid": "mediumturquoise",
        "RoundSpermatid": "skyblue",
        "LateSpermatid": "steelblue",    
        "ElongatingSpermatid": "mediumpurple",
        "ElongatedSpermatid": "indigo"
    }) #, save = "MdGermClusters_detailed_2.png")


In [ ]:
new_labels_som = {"0": "JTelocyte",
                  ...
                "11": "Leydig"}
#'JTelocyte', 'KSertoli', 'Leydig', 'Macrophage', 'NEndothelial', 'OPeritubular'

# Cluster Cell Numbers

In [ ]:
adata_germ

In [ ]:
sum(adata_germ.obs["new_labels_germ"] == "Spermatogonia")

In [ ]:
sum(adata_germ.obs["new_labels_germ"] == "Spermatocyte")

In [ ]:
sum(adata_germ.obs["new_labels_germ"] == "RoundSpermatid")

In [ ]:
sum(adata_germ.obs["new_labels_germ"] == "ElongatingSpermatid")

In [ ]:
sum(adata_germ.obs["new_labels_germ_2"] == "Spermatocyte")

In [ ]:
sum(adata_germ.obs["new_labels_germ_2"] == "LateSpermatocyte")

In [ ]:
sum(adata_germ.obs["new_labels_germ_2"] == "RoundSpermatid")

In [ ]:
sum(adata_germ.obs["new_labels_germ_2"] == "LateSpermatid")

In [ ]:
sum(adata_germ.obs["new_labels_germ_2"] == "ElongatingSpermatid")

In [ ]:
sum(adata_germ.obs["new_labels_germ_2"] == "ElongatedSpermatid")

In [ ]:
adata_som

# Cluster Markers for labeled clusters

In [ ]:
myslice = "new_labels_germ"

# conduct the 3 main tests on cluster IDs
sc.tl.rank_genes_groups(adata_germ, myslice, method='logreg', key_added="rank_genes_groups_logreg")
sc.tl.rank_genes_groups(adata_germ, myslice, method='t-test', key_added="rank_genes_groups_t-test")
sc.tl.rank_genes_groups(adata_germ, myslice, method='wilcoxon', key_added="rank_genes_groups_wilcoxon")

In [ ]:
# Write Cluster Markers

myslice = "new_labels_germ"
bonus = "Md_GermCells_ThreeReps_coarse_"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:  
    # Wilcoxon marker identification
    result = adata_germ.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

In [ ]:
# Write Cluster Markers

myslice = "new_labels_germ"
bonus = "Md_GermCells_ThreeReps_coarse_Top200_"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:  
    # Wilcoxon marker identification
    result = adata_germ.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(200)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

In [ ]:
myslice = "new_labels_germ_2"

# conduct the 3 main tests on cluster IDs
sc.tl.rank_genes_groups(adata_germ, myslice, method='logreg', key_added="rank_genes_groups_logreg")
sc.tl.rank_genes_groups(adata_germ, myslice, method='t-test', key_added="rank_genes_groups_t-test")
sc.tl.rank_genes_groups(adata_germ, myslice, method='wilcoxon', key_added="rank_genes_groups_wilcoxon")

In [ ]:
# Write Cluster Markers

myslice = "new_labels_germ_2"
bonus = "Md_GermCells_ThreeReps_fine_"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:  
    # Wilcoxon marker identification
    result = adata_germ.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

In [ ]:
# Write Cluster Markers

myslice = "new_labels_germ_2"
bonus = "Md_GermCells_ThreeReps_fine_Top200"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:  
    # Wilcoxon marker identification
    result = adata_germ.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(200)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

In [ ]:
myslice = "new_labels_som"

# conduct the 3 main tests on cluster IDs
sc.tl.rank_genes_groups(adata_som, myslice, method='logreg', key_added="rank_genes_groups_logreg")
sc.tl.rank_genes_groups(adata_som, myslice, method='t-test', key_added="rank_genes_groups_t-test")
sc.tl.rank_genes_groups(adata_som, myslice, method='wilcoxon', key_added="rank_genes_groups_wilcoxon")

In [ ]:
# Write Cluster Markers

myslice = "new_labels_som"
bonus = "Md_SomCells_ThreeReps_"

# mkdir for output if not exist already
outdir = os.path.abspath(os.path.join(os.getcwd(), "outs", "dge"))
if os.path.exists(outdir) == False:
    os.makedirs(outdir)

with pd.ExcelWriter(os.path.join(outdir, 'cluster_markers_' + bonus + myslice + '.xlsx')) as writer:  
    # Wilcoxon marker identification
    result = adata_som.uns['rank_genes_groups_wilcoxon']
    groups = result['names'].dtype.names
    markers = pd.DataFrame({group + '_' + key[:6]: result[key][group] for group in groups for key in ['names', 'pvals_adj']}).head(100)
    markers.to_excel(writer, sheet_name = "markers_all_wilcoxon")

# Aside for saving and reloading data

In [ ]:
#save germ cell subset as file
adata_germ.write_h5ad("Opossum_germcells_ThreeRep.h5ad")

In [ ]:
adata_som.write_h5ad("Opossum_Somcells_ThreeRep.h5ad")

In [ ]:
adata_sub.write_h5ad("Opossum_Subcells_ThreeRep.h5ad")

In [ ]:
# read h5ad back if needed
adata_germ = anndata.read_h5ad("path/to/Opossum_germcells_ThreeRep.h5ad")

In [ ]:
adata_som = anndata.read_h5ad("path/to/Opossum_Somcells_ThreeRep.h5ad")

# Dot Plots

In [ ]:
#Dotplot for marker genes
sc.pl.dotplot(adata_germ, marker_genes_comb_germ, groupby='leiden_1.4', swap_axes=True, cmap='Blues')

#, save = "Md_GermCell_Top5_celltype_newspermatocyte.png"

In [ ]:
sc.pl.dotplot(adata_germ, Top5_Gene_list, groupby='new_labels_germ', swap_axes=True, standard_scale = 'var', cmap='BrBG')
sc.pl.dotplot(adata_germ, Top5_Gene_list, groupby='new_labels_germ', swap_axes=True, standard_scale = 'var', cmap='Blues', save = "Md_GermCell_Top5_ThreeReps_dot_Blues_scaled.png")
             
#, save = "Md_GermCell_Top5_ThreeReps_dot_Blues_scaled.png"

#, save = "Md_GermCell_Top5_notrans_dot_BrBG_scaled.png"



In [ ]:
sc.pl.heatmap(adata_germ, clust_genes_germ_list, groupby='new_labels_germ', cmap='Blues', standard_scale='obs', swap_axes=True)
#, save = "Md_Germ_ThreeReps_Heatmap_scaled_blues.png"

# Pseudobulk


In [ ]:
import decoupler as dc
# Import DESeq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats


In [ ]:
adata_germ_2 = adata_germ

In [ ]:
adata_germ.layers['counts']

In [ ]:
adata_germ_2.X = np.round(adata_germ_2.X)
adata_germ_2.layers['counts'] = adata_germ_2.X

In [ ]:
# Normalize and log-transform
sc.pp.normalize_total(adata_germ_2, target_sum=1e4)

adata_germ_2.layers['normalized'] = adata_germ_2.X

In [ ]:
# Get pseudo-bulk profile
pdata = dc.get_pseudobulk(adata_germ,
                          sample_col='sample_labels',
                          groups_col='new_labels_germ',
                          mode='sum',
                          layer = 'counts',
                          min_cells=10,
                          min_counts=1000
                         )
pdata

In [ ]:
adata_germ.layers

In [ ]:
# Get pseudo-bulk profile
pdata2 = dc.get_pseudobulk(adata_germ_2,
                          sample_col='sample_labels',
                          groups_col='new_labels_germ_2',
                          mode='sum',
                          layer = 'counts',
                          min_cells=10,
                          min_counts=1000
                         )
pdata2

In [ ]:
pdata.obs['psbulk_counts']

In [ ]:
pdata2.obs['psbulk_counts']

In [ ]:
dc.plot_psbulk_samples(pdata, groupby=['sample_labels', 'new_labels_germ'], figsize=(11, 3))

In [ ]:
dc.plot_psbulk_samples(pdata2, groupby=['sample_labels', 'new_labels_germ_2'], figsize=(11, 3))

In [ ]:
dc.plot_filter_by_expr(pdata, group='new_labels_germ', min_count=10, min_total_count=15)

In [ ]:
dc.plot_filter_by_expr(pdata2, group='new_labels_germ_2', min_count=10, min_total_count=15)

In [ ]:
pdata.X

In [ ]:
pdata.X = np.round(pdata.X)

In [ ]:
pdata2.X = np.round(pdata2.X)

In [ ]:
# Build DESeq2 object
dds = DeseqDataSet(
    adata=pdata,
    design_factors='new_labels_germ',
    refit_cooks=True,
    n_cpus=8,
)

dds2 = DeseqDataSet(
    adata=pdata2,
    design_factors='new_labels_germ_2',
    refit_cooks=True,
    n_cpus=8,
)


In [ ]:
# Compute LFCs
dds.deseq2()

In [ ]:
dds2.deseq2()

In [ ]:
stat_res = DeseqStats(dds, contrast=["new_labels_germ", 'Spermatogonia', 'Spermatocyte'], n_cpus=8)
stat_res2 = DeseqStats(dds2, contrast=["new_labels_germ_2", 'Spermatogonia', 'Spermatocyte'], n_cpus=8)

In [ ]:
stat_res.summary()
stat_res2.summary()

# Making a table with means

In [ ]:
res = pd.DataFrame(columns= adata_germ.var_names, index=adata_germ.obs['new_labels_germ'].cat.categories)                                                                                                 

for clust in adata_germ.obs.new_labels_germ.cat.categories: 
    res.loc[clust] = adata_germ[adata_germ.obs['new_labels_germ'].isin([clust]),:].X.mean(0)
    

res2 = pd.DataFrame(columns= adata_germ.var_names, index=adata_germ.obs['new_labels_germ_2'].cat.categories)                                                                                                 

for clust in adata_germ.obs.new_labels_germ_2.cat.categories: 
    res2.loc[clust] = adata_germ[adata_germ.obs['new_labels_germ_2'].isin([clust]),:].X.mean(0)
    

In [ ]:
adata_germ.obs['new_labels_germ_2'].cat.categories

In [ ]:
df = res.transpose()
df2 = res2.transpose()

In [ ]:
df.to_excel(excel_writer = "Md_cluster_averages_ThreeReps.xlsx")
df2.to_excel(excel_writer = "Md_cluster_averages_ThreeReps_fine.xlsx") 

In [ ]:
res_p = pd.DataFrame(columns= pdata.var_names, index=adata_germ.obs['new_labels_germ'].cat.categories)                                                                                                 

for clust in adata_germ.obs.new_labels_germ.cat.categories: 
    res_p.loc[clust] = pdata[pdata.obs['new_labels_germ'].isin([clust]),:].X.mean(0)
    

res_p2 = pd.DataFrame(columns= pdata2.var_names, index=adata_germ.obs['new_labels_germ_2'].cat.categories)                                                                                                 

for clust in adata_germ.obs.new_labels_germ_2.cat.categories: 
    res_p2.loc[clust] = pdata2[pdata2.obs['new_labels_germ_2'].isin([clust]),:].X.mean(0)

In [ ]:
df_p = res_p.transpose()
df_p2 = res_p2.transpose()

In [ ]:
df_p.to_excel(excel_writer = "Md_pdata_ThreeReps_cluster_averages.xlsx")
df_p2.to_excel(excel_writer = "Md_pdata2_ThreeReps_cluster_averages.xlsx")  

# Species Differences Validation

In [ ]:
chinpy.pp.search(adata_germ, 'ENSMODG00000000396', startswith=True)

In [ ]:
SpecDiff = ['HORMAD1 (ENSMODG00000018807)', 'PIWIL2 (ENSMODG00000009482)', 'HEXB (ENSMODG00000001872)', 'CRABP1 (ENSMODG00000011710)', 'COL3A1 (ENSMODG00000010811)']


In [ ]:
# plot marker genes

fig, axes = plt.subplots(2,3, figsize=(4*3,4*2))
for j, ax in zip(SpecDiff, axes.flatten()):
    sc.pl.umap(adata_germ, color=chinpy.pp.search(adata_germ, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)
plt.savefig("Md_GermCells_SpeciesDiff_ThreeReps.png")      
fig.tight_layout()

In [ ]:
#violin plots for germ cell marker genes with coarse label clusters
sc.pl.violin(adata_germ, ['CRABP1 (ENSMODG00000011710)', 'SOHLH1 (ENSMODG00000029683)', 
                'KIT (ENSMODG00000020671)', 'DMRT1 (ENSMODG00000003401)',  'UCHL1 (ENSMODG00000020568)'] , groupby='new_labels_germ', layer="sqrtcpm_magic")

sc.pl.violin(adata_germ, ['STRA8 (ENSMODG00000014081)', 'PIWIL1 (ENSMODG00000015619)', 'SYCP1 (ENSMODG00000004800)', 'SYCP2 (ENSMODG00000016617)', 'CCNA1 (ENSMODG00000009993)'] , groupby='new_labels_germ', layer="sqrtcpm_magic")

sc.pl.violin(adata_germ, ['AURKA (ENSMODG00000016435)', 
                'ZPBP (ENSMODG00000009302)', 'TMEM144 (ENSMODG00000001956)', 
                'SPACA1 (ENSMODG00000018292)'] , groupby='new_labels_germ', layer="sqrtcpm_magic")
    
sc.pl.violin(adata_germ, ['TNP1 (ENSMODG00000024866)', 'TSSK6 (ENSMODG00000023787)',  
                'PRM1 (ENSMODG00000005135)', 'PIWIL2 (ENSMODG00000009482)'] , groupby='new_labels_germ', layer="sqrtcpm_magic")


In [ ]:
# plot germ cell marker genes with coarse cluster labels
fig, axes = plt.subplots(5,3, figsize=(4*3,4*5))
for j, ax in zip(marker_genes_germ, axes.flatten()):
    sc.pl.umap(adata_germ, color=chinpy.pp.search(adata_germ, j, startswith=True), layer="sqrtcpm_magic", title=j, legend_loc="on data", show=False, ax=ax)     
fig.tight_layout()

In [ ]:
#violin plots for somatic marker genes 
sc.pl.violin(adata_som, ['DCN (ENSMODG00000009654)', 'ENSMODG00000020670 (ENSMODG00000020670)', 'TAGLN (ENSMODG00000013744)',
                         'ACTA2 (ENSMODG00000007394)'] , groupby='new_labels_som', layer="sqrtcpm_magic") #, save = "Md_Som_DevCell_Violin1.png")


sc.pl.violin(adata_som, ['RAMP2 (ENSMODG00000015038)', 'PECAM1 (ENSMODG00000025673)', 'CD74 (ENSMODG00000028756)', 
                         'TYROBP (ENSMODG00000014057)'] , groupby='new_labels_som', layer="sqrtcpm_magic") #, save = "Md_Som_DevCell_Violin2.png")

sc.pl.violin(adata_som, ['CLU (ENSMODG00000015960)', 'SOX9 (ENSMODG00000006526)'] , groupby='new_labels_som', layer="sqrtcpm_magic") #, save = "Md_Som_DevCell_Violin3.png")

sc.pl.violin(adata_som, ['FSHR (ENSMODG00000001267)', 'STAR (ENSMODG00000010748)', 'CYP11A1 (ENSMODG00000009689)', 
                         'CYP17A1 (ENSMODG00000011205)'] , groupby='new_labels_som', layer="sqrtcpm_magic") #, save = "Md_Som_DevCell_Violin4.png")
